# Credit Card Recommender

This notebook will demo the credit card recommendation workflow utilizing GPT. 

## Preprocessing
1. Read in transaction history (CSV) -> convert to JSON string
2. Read credit card data as JSON

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pprint import pprint
import json

from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from mcc_map import convert_mcc_to_category

In [3]:
# Import transaction data
tx_df = pd.read_csv('dummy-data/tx-data.csv')
convert_mcc_to_category(tx_df)
tx_df.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,grocery,NaN,No
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,grocery,NaN,No
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,grocery,NaN,No
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,retail,NaN,No
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,retail,NaN,No


In [4]:
# Group the transactions by category
tx_df["Amount"] = tx_df["Amount"].replace("[\$,]", "", regex=True).astype(float)

tx_grouped = tx_df.groupby("MCC")["Amount"].sum().sort_values(ascending=False)

In [10]:
# Import credit card data
with open("dummy-data/credit-card-data.json", "rb") as file:
    credit_card_json = json.load(file)

In [5]:
# Import user info
with open("dummy-data/user-info.json", "rb") as file:
    user_info = json.load(file)

## Connect to OpenAI API

In [6]:
client = OpenAI()

pprint(client.models.list().data)

[Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
 Model(id='babbage-002', created=1692634615, object='model', owned_by='system'),
 Model(id='davinci-002', created=1692634301, object='model', owned_by='system'),
 Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'),
 Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai

### Let's try feeding both JSONs directly to the model

### Quick demo

In [23]:
llm = ChatOpenAI(model="chatgpt-4o-latest")
output_parser = StrOutputParser()

In [93]:
rec_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a world-class expert in credit cards and you specialize in making credit card recommendations."),
    (
        "user", """
        Here is the user's financial information:
        ```json
        {user_info}
        ```
        
        Here is the user's transaction history:
        ```json
        {transactions}
        ```
        
        Here is our database of credit card information:
        ```json
        {credit_cards}
        ```
        
        Please recommend up to 4 credit cards that maximizes the savings/value across all categories. Assume the user will use all the cards you recommend, so they cannot be redundant. Ensure that you only recommend cards that the user meets the minimum credit score requirement for. 
        
        Format your answer as follows:
        1. Think through the problem step-by-step. Talk about possible factors to consider when choosing a credit card as a customer
        2. Taking all these factors into account, explain your reasoning step-by-step, which ultimately leads to your recommendations. Ensure to explain your reasoning for why you ranked some cards higher/lower than others.
        3. Rank your recommendations in ascending order by most recommended to least recommended. Output your recommendations in a list along with their value proposition like this:
        
        <answer>
        1. <card name>: <reasoning>
        2. <card name>: <reasoning>
        ...
        </answer>
        """)
])

In [94]:
rec_chain = rec_prompt | llm | output_parser

In [95]:
tx_grouped = tx_df.groupby("MCC")["Amount"].sum().sort_values(ascending=False)

# income = 100000

output = rec_chain.invoke({
    "transactions": tx_grouped,
    # "income": income,
    "credit_cards": credit_card_json,
})

In [103]:
tx_grouped

MCC
onlineDining     $9.92$6.51$4.73$8.40$8.55$5.76$7.10$6.44$6.56$...
dining           $76.07$80.18$12.76$92.30$76.37$80.65$80.05$87....
other            $57.10$29.34$65.50$160.00$1.13$53.09$51.98$82....
gas              $53.91$110.37$62.06$-99.00$99.00$145.50$153.36...
transit          $36.82$21.10$100.34$31.68$31.40$89.12$29.38$81...
rentalCar        $331.00$150.59$-331.00$167.05$380.00$-380.00$2...
entertainment    $22.56$29.33$29.95$227.90$32.23$29.35$32.56$28...
hotel            $211.00$149.45$-211.00$255.00$-255.00$147.41$2...
streaming        $144.90$210.96$185.66$188.40$192.46$179.71$222...
grocery          $134.09$38.48$120.34$93.84$123.50$61.72$117.05...
retail           $128.95$104.71$86.19$45.30$147.45$56.42$36.73$...
travel           $100.00$-100.00$164.52$393.00$-393.00$161.78$4...
Name: Amount, dtype: object

In [96]:
import re
from rich.console import Console
from rich.markdown import Markdown

sections = re.split(r"###", output)

console = Console()

for section in sections:
    if section.strip():
        markdown_content = Markdown(f"### {section.strip()}")
        console.print(markdown_content)

Step 1: Analyze User's Spending Patterns                                      

To make the best credit card recommendations, it's critical to first analyze the user's spending habits, which will
highlight the categories where they spend the most and would benefit from maximizing rewards. Here's the breakdown 
of the user's transaction history:                                                                                 

 • Online Dining: Frequent, small transactions around $6-10, indicating regular spending.                          
 • Dining: High spending in dining (~$80 per transaction), showing a significant part of their budget.             
 • Other: Mixed spending on various items.                                                                         
 • Gas: Regular spending on gas.                                                                                   
 • Transit: Medium-level spending on transit.                                                                      
 • Rental Car: Frequent car rentals, with high costs.                                                              
 • Entertainment: Recurring spending on entertainment.                                                             
 • Hotel: Occasional but large hotel transactions.                                                                 
 • Streaming: Consistent spending on streaming services.                                                           
 • Grocery: Regular grocery shopping, some high expenditure.                                                       
 • Retail: Regular spending on retail, likely for general purchases.

Step 2: Factors to Consider                                            

When choosing a credit card, the following factors should be considered:                                           

 1 Rewards Categories: The cards should offer high rewards in the categories the user spends the most (dining,     
   travel, gas, streaming, groceries, etc.).                                                                       
 2 Sign-up Bonuses: Large sign-up bonuses with reasonable spending thresholds can provide immediate value.         
 3 Annual Fees: Balancing the annual fees with the expected rewards and benefits is essential. Ideally, the value  
   of the rewards should outweigh the annual fee.                                                                  
 4 Additional Benefits: Benefits such as travel insurance, rental car insurance, and foreign transaction fee       
   waivers can add significant value.                                                                              
 5 No Redundancy: Since the user will use all the recommended cards, they should complement each other without     
   overlapping too much in rewards categories.

Step 3: Reasoning Step-by-Step

# Card 1: American Express Gold Card                                        

 • Why Recommended: This is one of the best cards for dining (4x points per dollar) and grocery (4x points per     
   dollar), which are two major spending categories for this user. The card also gives 3x points on travel, which  
   covers hotel and transit spending. The $120 annual dining credit, along with Uber Cash, provides extra value.   
   The sign-up bonus of 80,000 points with a $6,000 spend in 6 months is attractive.                               
 • Annual Fee: $325, but the rewards on dining and groceries will likely outweigh this cost, especially with the   
   dining credit.

# Card 2: Chase Sapphire Preferred Card                                      

 • Why Recommended: This card offers 3x points on dining and 5x points on travel. The user spends significantly on 
   travel-related categories (hotel, transit, rental car), so the 5x travel multiplier will provide significant    
   value. The card also has a 60,000-point sign-up bonus after spending $4,000 in 3 months, which is achievable    
   based on the user's history. Additional benefits such as primary rental car insurance and travel credits make   
   this card an excellent choice for travel-related purchases.                                                     
 • Annual Fee: $95, a reasonable fee given the strong travel and dining rewards.

# Card 3: Discover it Cash Back                                          

 • Why Recommended: Discover it Cash Back offers 5% cash back in rotating categories (up to $1,500) each quarter,  
   which can align with the user's spending on gas, groceries, or retail. This is a no-annual-fee card, meaning any
   cash back earned is pure value. The cashback match at the end of the first year effectively doubles the rewards,
   which can offer a significant return.                                                                           
 • Annual Fee: None, making it a highly cost-effective card for rotating categories.

# Card 4: Capital One Venture Rewards Credit Card                                 

 • Why Recommended: This card offers 5x miles on travel (hotel, rental car, vacation rental), which matches some of
   the user's highest expenses. It also has a 75,000-mile bonus with a $4,000 spend in 3 months, making it a strong
   travel card. This card complements the Chase Sapphire Preferred by covering more rental car related purchases   
   and offering a flat 2x miles on all other purchases, which can cover spending in miscellaneous categories like  
   transit, retail, and entertainment.                                                                             
 • Annual Fee: $95, but the rewards on travel and 2x miles on everything else will more than cover this fee.

Step 4: Final Recommendations

## Use OpenAI's structured outputs

In [7]:
sys_prompt = """You are a world-class credit card advisor with deep expertise in personal finance, rewards optimization, and credit card benefits. Your role is to provide personalized credit card recommendations that maximize value while considering:
- Credit score requirements and approval odds
- Annual fees vs. potential rewards value
- Complementary benefits across multiple cards
- User's spending patterns and lifestyle
- Sign-up bonuses and welcome offers
- Credit issuer application rules (like Chase 5/24)
- Long-term value proposition beyond first year

Maintain a balanced perspective between optimizing rewards and responsible credit management. Be transparent about annual fees and potential drawbacks of recommended cards.
"""

rec_prompt = """Here is the user's financial information:
    ```json
    {user_info}
    ```
    
    Here is the user's transaction history:
    ```json
    {transactions}
    ```
    
    Here is our database of credit card information:
    ```json
    {credit_cards}
    ```
    
    Please recommend up to 4 credit cards that maximize the savings/value across all categories. Assume the user will use all the cards you recommend, so they cannot be redundant. Ensure that you only recommend cards that the user meets the minimum credit score requirement for.
    
    Follow this reasoning process to generate your response:
    1. **Step-by-Step Reasoning**: Begin by analyzing the user’s financial profile and spending patterns, considering factors like lifestyle, spending habits, credit score, and long-term card value. Mention key factors that influence credit card selection.
    2. **Rationale for Rankings**: Discuss the logic behind why certain cards are ranked higher or lower based on the user's needs, approval odds, potential value, and card benefits.

    Format your answer using the following pydantic models:

    ```python
    class UserInfo(BaseModel):
        global_reasoning: str  # An overview of the factors considered for all recommendations
        recommendations: list[CreditCardRec]  # A list of credit card recommendations

    class CreditCardRec(BaseModel):
        rec_reasoning: str  # Explanation for recommending this specific card
        card_name: str  # Name of the credit card being recommended
    ```

    ### Output Format
    - `UserInfo.global_reasoning`: A paragraph summarizing key factors considered across all cards.
    - `UserInfo.recommendations`: A list of `CreditCardRec` objects, each containing:
        - `rec_reasoning`: Step-by-step reasoning for why this specific card was chosen.
        - `card_name`: Name of the credit card.
    
    Please use this structured format in your response.
    """

In [11]:
from pydantic import BaseModel

class CreditCardRec(BaseModel):
    rec_reasoning: str
    card_name: str
    
class UserInfo(BaseModel):
    global_reasoning: str
    recommendations: list[CreditCardRec]


completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": sys_prompt
        },
        {
            "role": "user",
            "content": rec_prompt.format(user_info=user_info, transactions=tx_grouped, credit_cards=credit_card_json)
        },
    ],
    response_format=UserInfo,
)

In [13]:
rec = completion.choices[0].message.parsed
pprint(rec.model_dump())

{'global_reasoning': "The user's financial profile indicates a high credit "
                     'score of 800, making them eligible for any card in the '
                     'database. With an income of $100,000 and high spending, '
                     'particularly in grocery, retail, and gas, cards that '
                     'offer high rewards rates in these categories would '
                     'optimize value. Given a willingness to pay up to $800 in '
                     'annual fees, premium cards with significant travel and '
                     'dining benefits should be considered. However, balance '
                     'must be struck to avoid redundancy while maximizing '
                     'rewards. Moreover, signup bonuses play a role in the '
                     "short-term value, and the card's credit issuer rules, "
                     "like Chase's 5/24 rule, affect approval odds.",
 'recommendations': [{'card_name': 'Blue Cash Preferred Card from AmEx',

## Calculate the user's savings using the recommended cards

In [14]:
tx_grouped

MCC
grocery          497988.79
retail           487190.51
other            253515.79
gas              185779.92
dining            90767.45
travel            49914.39
streaming         24677.00
hotel             21504.94
entertainment      7487.52
onlineDining       1621.93
transit            1563.74
rentalCar           979.71
Name: Amount, dtype: float64

In [15]:
credit_card_json

{'creditCards': [{'cardName': 'Chase Sapphire Preferred Card',
   'cardType': 'Credit Card',
   'issuer': 'Chase',
   'annualFee': 95,
   'APR': '20.99% - 27.99%',
   'rewards': {'pointsPerDollar': {'travel': 5,
     'dining': 3,
     'onlineGrocery': 3,
     'streaming': 3,
     'other': 1},
    'signUpBonus': {'points': 60000,
     'minimumSpend': 4000,
     'timeFrameMonths': 3}},
   'benefits': ['$300 in travel credits in the first year',
    'Primary rental car insurance',
    'No foreign transaction fees',
    '$150 in additional partnership benefit value',
    '1:1 point transfer with partners',
    'Travel and purchase coverage'],
   'creditCardScoreMin': 650,
   'creditCardScoreMax': 850,
   'linkToApply': 'https://creditcards.chase.com/rewards-credit-cards/sapphire/preferred',
   'countryOfOrigin': 'USA'},
  {'cardName': 'Capital One Venture Rewards Credit Card',
   'cardType': 'Credit Card',
   'issuer': 'Capital One',
   'annualFee': 95,
   'APR': '19.99% - 29.74%',
   'rew